In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install chromadb langchain tiktoken sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 289.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

# InstructorEmbedding
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

import pandas as pd
import numpy as np

C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [5]:
# Daten einlesen und filtern
df = pd.read_parquet("/content/drive/MyDrive/Masterthesis/2023-06-23_gesamtdaten_rhein_neckar.parquet")
df = df[df['kommune'].isin(['Frankenthal', 'Schwetzingen', 'Lampertheim', 'Kreis Bergstraße'])]
df = df[(df['Gesamt'] == True) | (df['stichwort_verkehr'] == True)]
df = df[df['token_laenge'] > 1]
df = df[['text', 'index_doc', 'datum_string','kommune', 'doc_name', 'beschlussstatus']]
df = df.fillna('k.A.')

In [6]:
# Daten laden
loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=1000,
                                               chunk_overlap=200)

texts = text_splitter.split_documents(documents)

# Get Embeddings

In [2]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [10]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '/content/drive/MyDrive/Masterthesis/chromadb_municipalities_instrucstor_xl'

## Here is the nmew embeddings being used

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
persist_directory = '/data/embeddings/chromadb_municipalities_instrucstor_xl'

vectordb = Chroma(persist_directory=persist_directory, embedding_function=instructor_embeddings)

# HF Instructor Embeddings

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 45})

In [ ]:
# create the chain to answer questions

llm = ChatOpenAI(temperature = 0, model="gpt-3.5-turbo-16k-0613")

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  # verbose=True
                                  )
qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template = '''
You are an expert for transportation and mobility research. Don't make up an answer and always answer truthfully.
You speak and answer in German.
----------------
{context}'''

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(f"{source.metadata['kommune']} - {source.metadata['datum_string']} - {source.metadata['doc_name']} - {source.metadata['index_doc']}")

def question(query):
  llm_response = qa_chain(query)
  process_llm_response(llm_response)

In [ ]:
docs = retriever.get_relevant_documents("Was würde in der Stadt Frankenthal für den Radverkehr getan?")